[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shang-vikas/series1-coding-exercises/blob/main/exercises/blog-01/exercise-01.ipynb)

In [10]:

# Install required packages using the kernel's Python interpreter
import sys
import subprocess
import importlib

def install_if_missing(package, import_name=None):
    """Install package if it's not already installed."""
    if import_name is None:
        import_name = package
    try:
        importlib.import_module(import_name)
        print(f"✓ {package} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} installed successfully")

# Install required packages
install_if_missing("numpy")
install_if_missing("scikit-learn", "sklearn")
install_if_missing("pandas")

✓ numpy is already installed
✓ scikit-learn is already installed
Installing pandas...
  Using cached pandas-2.3.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.3-cp310-cp310-macosx_11_0_arm64.whl (10.8 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.3-py2.py3-none-any.whl (348 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]
✓ pandas installed successfully



[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip


In [11]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_auc_score
)

# Load dataset
data = load_breast_cancer()

# Convert to DataFrame for readability
df = pd.DataFrame(data.data, columns=data.feature_names)
df["target"] = data.target

print("First 5 rows of the dataset:\n")
print(df.head())

print("\nDataset shape:", df.shape)
print("\nClass distribution:\n")
print(df["target"].value_counts())

# Split data
X = df.drop("target", axis=1)
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Scale for SVM (important)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM (Linear)": SVC(kernel="linear", probability=True, random_state=42)
}

for name, model in models.items():
    
    if "SVM" in name:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_prob = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1]
    
    print(f"\n===== {name} =====")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred))
    print("ROC-AUC:", roc_auc_score(y_test, y_prob))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


First 5 rows of the dataset:

   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst texture  worst perimeter

# Understanding Classification Metrics

> **The difference between a practitioner and a button-pusher:** Understanding *why* metrics matter, not just *what* they calculate.

---

## 📊 The Foundation: Confusion Matrix

The **confusion matrix** is the fundamental truth table of predictions. Every other metric derives from it.

### Components

| Term | Definition | Example (Cancer Detection) |
|------|------------|----------------------------|
| **True Positive (TP)** | Correctly predicted positive | Correctly identified malignant tumor |
| **True Negative (TN)** | Correctly predicted negative | Correctly identified benign tumor |
| **False Positive (FP)** | Predicted positive, actually negative | False alarm: predicted malignant, but benign |
| **False Negative (FN)** | Predicted negative, actually positive | **Most dangerous:** Missed cancer diagnosis |

### Why It Matters

In medical diagnosis, **false negatives are life-threatening**. This isn't just mathematics—it's a missed diagnosis that could cost a life.

---

## 🎯 Core Metrics Explained

### 1. Accuracy

**Formula:** `Accuracy = (TP + TN) / (TP + TN + FP + FN)`

**What it answers:** *"How often is the model correct overall?"*

**The Problem:**
- Can be **misleading with imbalanced classes**
- Example: If 95% of patients are healthy, a model that always predicts "healthy" achieves 95% accuracy
- **That model is useless**—it never catches cancer!

**When to use:** Balanced datasets where all error types have similar cost.

---

### 2. Precision

**Formula:** `Precision = TP / (TP + FP)`

**What it answers:** *"When the model says 'malignant,' how often is it correct?"*

**Interpretation:**
- **High precision** = Few false alarms
- **Low precision** = Too many false positives (unnecessary anxiety, tests, costs)

**When to optimize:** Spam detection, fraud detection (where false alarms are costly)

---

### 3. Recall (Sensitivity)

**Formula:** `Recall = TP / (TP + FN)`

**What it answers:** *"Of all the actual malignant cases, how many did we catch?"*

**Interpretation:**
- **High recall** = Catching most positive cases
- **Low recall** = Missing too many positive cases

**When to optimize:** Medical diagnosis, search engines (missing results is worse than extra results)

> **Key Insight:** In cancer detection, **recall often matters more than precision**. Missing cancer is worse than ordering an unnecessary follow-up test.

---

### 4. F1 Score

**Formula:** `F1 = 2 × (Precision × Recall) / (Precision + Recall)`

**What it is:** Harmonic mean of precision and recall

**Why harmonic mean?**
- Penalizes **extreme imbalance** between precision and recall
- If one is high and the other is low, F1 drops sharply
- Forces a **balanced trade-off**

**When to use:** When you need a single metric that balances both precision and recall

---

### 5. ROC-AUC

**What it stands for:** Receiver Operating Characteristic - Area Under the Curve

**What it measures:** How well the model **separates classes** across all possible classification thresholds

**Scale:**
- **1.0** → Perfect class separation
- **0.5** → Random guessing (no better than coin flip)
- **< 0.5** → Model is worse than random (likely inverted)

**Key Properties:**
- **Threshold-independent** — evaluates ranking ability, not just final predictions
- Measures the model's ability to **distinguish** between classes overall

**What it answers:** *"How well does the model distinguish malignant from benign across all possible decision thresholds?"*

---

## 🤖 Model Characteristics

### Decision Tree
- ✅ **Interpretable** (visual flowchart-like structure)
- ✅ **Easy to explain** to non-technical stakeholders
- ⚠️ **Prone to overfitting** (memorizes training data)
- ⚠️ **High variance** (small data changes → different tree)

### Random Forest
- ✅ **Reduces variance** (ensemble of many trees)
- ✅ **Strong accuracy and AUC** (often top performer)
- ✅ **Handles non-linearity** well
- ⚠️ **Less interpretable** (black box of many trees)
- ⚠️ **Slower inference** than single tree

### SVM (Support Vector Machine)
- ✅ **Strong theoretical foundation** (maximizes margin)
- ✅ **Excellent in high-dimensional spaces** (like our 30-feature medical data)
- ✅ **Memory efficient** (only uses support vectors)
- ⚠️ **Sensitive to feature scaling** (requires normalization)
- ⚠️ **Less interpretable** (complex decision boundaries)
- ⚠️ **Slow on large datasets**

---

## 💡 The Principal Engineer's Perspective

### The Real Point

**Machine learning is not about picking "the best model."**

It's about **choosing the right trade-off** for your specific problem and business context.

### Context Matters

| Problem Domain | Priority Metric | Why |
|----------------|----------------|-----|
| **Cancer Detection** | **Recall** | Missing cancer is catastrophic |
| **Spam Detection** | **Precision** | False alarms annoy users |
| **Fraud Detection** | **F1 Score** | Balance catching fraud vs. false alarms |
| **Search Engines** | **Recall** | Missing relevant results is worse than extra results |
| **Financial Trading** | **Custom Cost Function** | False positives and negatives have different dollar costs |

### Remember

> **Metrics are not just numbers—they encode priorities.**

Every metric you choose tells a story about what you value:
- High recall? You prioritize catching all positive cases.
- High precision? You prioritize avoiding false alarms.
- High F1? You want a balanced approach.

**Choose your metrics based on your problem's real-world consequences, not just mathematical convenience.**